In [104]:
#importing the necessary libraries

from googleapiclient.discovery import build
import pandas as pd
import numpy as np
import seaborn as sns

In [105]:
#setting the api_key

api_key ='#add the api key here'

In [106]:
#setting the channel id

channel_id = ['UCBJycsmduvYEL83R_U4JriQ'] #mkbhd main channel  

In [107]:
#setting the service

youtube = build('youtube','v3',developerKey = api_key)

## Function to get overall channel statistics

In [108]:
def get_channel_stats(youtube,channel_id):
    
    request = youtube.channels().list(
            part='snippet,contentDetails,statistics',
            id= ','.join(channel_id))
    response = request.execute()
    
    data = dict(Channel_name = response['items'][0]['snippet']['title'],
                Start_date = response['items'][0]['snippet']['publishedAt'],
                View_count = response['items'][0]['statistics']['viewCount'],
                Subscriber_count = response['items'][0]['statistics']['subscriberCount'],
                Video_count = response['items'][0]['statistics']['videoCount'],
                Upload_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
        
    return data

In [109]:
channel_details = get_channel_stats(youtube,channel_id)

In [110]:
channel_details = pd.DataFrame(channel_details,[0])

In [111]:
channel_details

,Channel_name,Start_date,View_count,Subscriber_count,Video_count,Upload_id
0,Marques Brownlee,2008-03-21T15:25:54Z,3236360054,16300000,1485,UUBJycsmduvYEL83R_U4JriQ


In [112]:
playlist_id = 'UUBJycsmduvYEL83R_U4JriQ'

In [113]:
playlist_id

'UUBJycsmduvYEL83R_U4JriQ'

## Function to get video ids

In [114]:
def get_video_ids(youtube, playlist_id):
    request = youtube.playlistItems().list(
        part="contentDetails",
        playlistId = playlist_id,
        maxResults=50)
    response = request.execute()
    
    video_ids =[]
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_token_id = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_token_id is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                    part="contentDetails",
                    playlistId = playlist_id,
                    maxResults=50,
                    pageToken = next_token_id)
            response = request.execute()
            
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_token_id = response.get('nextPageToken')
        
    return video_ids

In [115]:
video_ids = get_video_ids(youtube, playlist_id)

## Extracting useful details from the video ids

In [116]:
def every_video_statistics(youtube,video_ids):
    
    every_video_stats = []
    for i in range(0,len(video_ids),50):
        request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=','.join(video_ids[i:i+50]))
        response = request.execute()
        
        for video in response['items']:
            video_stats = dict(Title = video['snippet']['title'],
                               Published_on = video['snippet']['publishedAt'],
                               Description = video['snippet']['description'],
                               Duration = video['contentDetails']['duration'],
                               Definition = video['contentDetails']['definition'],
                               Caption = video['contentDetails']['caption'],
                               Tags = video['snippet'].get('tags'),
                               No_of_Views = video['statistics'].get('viewCount'),
                               No_of_likes = video['statistics'].get('likeCount'),
                               No_of_Comments = video['statistics'].get('commentCount')
                              )
            every_video_stats.append(video_stats)
    
    return every_video_stats

In [117]:
every_video_detail = every_video_statistics(youtube,video_ids)

In [118]:
every_mkbhd_video = pd.DataFrame(every_video_detail)

In [119]:
every_mkbhd_video.dtypes

Title             object
Published_on      object
Description       object
Duration          object
Definition        object
Caption           object
Tags              object
No_of_Views       object
No_of_likes       object
No_of_Comments    object
dtype: object

In [120]:
every_mkbhd_video['Published_on'] = pd.to_datetime(every_mkbhd_video['Published_on'])
every_mkbhd_video['Duration'] = pd.to_timedelta(every_mkbhd_video['Duration'])

In [121]:
every_mkbhd_video['No_of_Views'] = pd.to_numeric(every_mkbhd_video['No_of_Views'])
every_mkbhd_video['No_of_likes'] = pd.to_numeric(every_mkbhd_video['No_of_likes'])
every_mkbhd_video['No_of_Comments'] = pd.to_numeric(every_mkbhd_video['No_of_Comments'])

In [122]:
every_mkbhd_video.dtypes

Title                          object
Published_on      datetime64[ns, UTC]
Description                    object
Duration              timedelta64[ns]
Definition                     object
Caption                        object
Tags                           object
No_of_Views                   float64
No_of_likes                   float64
No_of_Comments                float64
dtype: object

In [123]:
every_mkbhd_video.head()

,Title,Published_on,Description,Duration,Definition,Caption,Tags,No_of_Views,No_of_likes,No_of_Comments
0,Lucid Air: How to One-up Tesla!,2022-11-11 19:27:05+00:00,Lucid Air beats Tesla's flagship at almost eve...,0 days 00:13:23,hd,true,"[Lucid Air, Lucid, Lucid vs Tesla, Lucid vs, M...",2743714.0,108827.0,5567.0
1,Dear Twitter!,2022-11-08 02:29:45+00:00,I can see you're going through a lot. But I ha...,0 days 00:14:30,hd,true,"[Twitter, dear twitter, MKBHD, Elon Musk, Elon...",4170058.0,183919.0,12366.0
2,The iPhone USB-C Law: Explained!,2022-11-02 17:58:32+00:00,All opinions in the video are my own. Armchair...,0 days 00:09:17,hd,true,"[mkbhd, USBC iPhone, iPhone USB, iPhone USBC, ...",4245084.0,170275.0,13432.0
3,Who Cares About the Metaverse?,2022-10-28 19:48:08+00:00,"My actual thoughts on the ""metaverse"" and VR a...",0 days 00:15:51,hd,true,None,7201966.0,227457.0,21216.0
4,The New iPad is Weird!,2022-10-25 23:14:46+00:00,The new 10th gen iPad is one of the weirdest r...,0 days 00:08:16,hd,true,"[iPad, iPad review, iPad 10th gen, 10th gen iP...",5941779.0,180072.0,13284.0


In [124]:
every_mkbhd_video['Year'] = every_mkbhd_video['Published_on'].dt.year
every_mkbhd_video['Month'] = every_mkbhd_video['Published_on'].dt.month
every_mkbhd_video['Day'] = every_mkbhd_video['Published_on'].dt.day
every_mkbhd_video['Time'] = every_mkbhd_video['Published_on'].dt.time

In [125]:
every_mkbhd_video

,Title,Published_on,Description,Duration,Definition,Caption,Tags,No_of_Views,No_of_likes,No_of_Comments,Year,Month,Day,Time
0,Lucid Air: How to One-up Tesla!,2022-11-11 19:27:05+00:00,Lucid Air beats Tesla's flagship at almost eve...,0 days 00:13:23,hd,true,"[Lucid Air, Lucid, Lucid vs Tesla, Lucid vs, M...",2743714.0,108827.0,5567.0,2022,11,11,19:27:05
1,Dear Twitter!,2022-11-08 02:29:45+00:00,I can see you're going through a lot. But I ha...,0 days 00:14:30,hd,true,"[Twitter, dear twitter, MKBHD, Elon Musk, Elon...",4170058.0,183919.0,12366.0,2022,11,8,02:29:45
2,The iPhone USB-C Law: Explained!,2022-11-02 17:58:32+00:00,All opinions in the video are my own. Armchair...,0 days 00:09:17,hd,true,"[mkbhd, USBC iPhone, iPhone USB, iPhone USBC, ...",4245084.0,170275.0,13432.0,2022,11,2,17:58:32
3,Who Cares About the Metaverse?,2022-10-28 19:48:08+00:00,"My actual thoughts on the ""metaverse"" and VR a...",0 days 00:15:51,hd,true,None,7201966.0,227457.0,21216.0,2022,10,28,19:48:08
4,The New iPad is Weird!,2022-10-25 23:14:46+00:00,The new 10th gen iPad is one of the weirdest r...,0 days 00:08:16,hd,true,"[iPad, iPad review, iPad 10th gen, 10th gen iP...",5941779.0,180072.0,13284.0,2022,10,25,23:14:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,Fraps HD Test in 1080p (18 WOS),2009-01-01 22:30:32+00:00,"***WATCH IN HD***\r\n\r\nAlso, check out my vi...",0 days 00:01:31,hd,false,"[fraps, hd, high, definition, 720, 1080, 18, w...",135416.0,1648.0,240.0,2009,1,1,22:30:32
1483,HP Pavilion dv7t Media Center Remote Overview,2009-01-01 19:43:47+00:00,This is my short review of the Media Center Re...,0 days 00:02:51,sd,false,"[hp, pavilion, dv7, laptop, 17, inch, desktop,...",4706337.0,236226.0,36885.0,2009,1,1,19:43:47
1484,High fps LG Voyager footage,2008-09-16 01:33:42+00:00,This video was uploaded directly from my LG Vo...,0 days 00:00:31,sd,false,"[mobile, video, cellphone, high, fps, footage,...",187336.0,2751.0,730.0,2008,9,16,01:33:42
1485,14 Year knock-down shot (11 Handicap),2008-05-30 20:27:17+00:00,"Me hitting a knock-down shot with a six iron, ...",0 days 00:00:41,sd,false,"[golf, tiger, woods, prodigy, amazing, analysi...",181370.0,2300.0,362.0,2008,5,30,20:27:17


In [129]:
every_mkbhd_video.to_csv('Mkbhd_dataset.csv')

#### credits: Thoufiq, https://www.youtube.com/c/techTFQ, without whose tutorial videos this project wouldn't have been a success.Tech TFQ is one of the most popular channels on youtube, that is a go to for any aspiring Data Analysts.   